## lesson-3

Библиотеки

In [308]:
import numpy as np

import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.8.0
TensorFlow 2.0 Hello World


In [4]:
x = tf.random.uniform([3, 3])

print("есть ли доступные GPU: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("тензор на GPU #0:  "),
print(x.device.endswith('GPU:0'))

есть ли доступные GPU: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
тензор на GPU #0:  
True


In [32]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 циклов: {:0.2f}ms".format(1000*result))

# исполнение CPU
print(" CPU работает:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# исполнение на GPU #0 , если доступен
if tf.config.experimental.list_physical_devices("GPU"):
  print(" какой GPU:")
  with tf.device("GPU:0"): # или GPU:1 , для 2х GPU, GPU:2 для 3х  и т.д.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

 CPU работает:
10 циклов: 117.00ms
 какой GPU:
10 циклов: 1.00ms


### 1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

In [43]:
boston = load_boston()

In [161]:
X_train, X_test, y_train, y_test = train_test_split(boston['data'], boston['target'], 
                                                                          test_size=0.3333, shuffle=True, random_state=42)

In [280]:
def nn(X_train, X_test, y_train, y_test, neuron = 100, layer = 2, optimizer='adam', 
                                   loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5)):
    
    metrics_train = []
    metrics_test = []
    
    for seed in seed_list:
        tf.random.set_seed(seed)
        model = Sequential(name='my_model')
        model.add(Dense(    neuron, activation='relu', input_shape=(13,)))  
        # входной слой + поносвязный слой с активацией ReLU
        for k in range(layer - 1):
          model.add(Dense(  neuron, activation='tanh')) 
          # Скрытый слой k из neuron_numb нейронов с активацией tanh

        model.add(Dense(1, activation='relu')) 
        # выходной слой из 10 нейронов (сколько классов, столько нейронов) + активация relu
        
        model.compile(
          optimizer=optimizer, # оптимизатор
          loss=loss, # функция потерь
          metrics=metrics, # метрика
        ) 
        
        with tf.device("GPU:0"):
            # Train the model.
            hh = model.fit(    
                X_train,
                y_train,
                epochs=15,
                batch_size=32, verbose=0
            )

            # Evaluate the model.
            eval_test = model.evaluate( 
                X_test,
                y_test, verbose=0
            )
        
        # print([hh.history[key][-1] for key in hh.history][1:], eval_test[1:])
        metrics_train.append([hh.history[key][-1] for key in hh.history][1:])
        metrics_test.append(eval_test[1:])
        
    return (dict(zip(list(hh.history.keys())[1:], np.mean(metrics_train, axis=0))), 
            dict(zip(list(hh.history.keys())[1:], np.mean(metrics_test, axis=0))))

In [281]:
%%time

nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                           loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5))

Wall time: 10.7 s


({'root_mean_squared_error': 8.286011695861816},
 {'root_mean_squared_error': 7.447188186645508})

При некоторых seed модель почему то расходится, я пока не нашёл что нужно настроить, чтобы она не расходилась, поэтому я решил зафиксировать seed = 0, 1, 2, 3, 4. Чтобы дальше эксперименты были +- честные.

### 2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [298]:
%%time

loss_list = [tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError(), 
             tf.keras.losses.MeanAbsolutePercentageError(), tf.keras.losses.MeanSquaredLogarithmicError(), 
             tf.keras.losses.CosineSimilarity(), tf.keras.losses.Huber(), tf.keras.losses.LogCosh()]

metrics_list = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError(), 
                tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError(),
                tf.keras.metrics.MeanSquaredLogarithmicError(), tf.keras.metrics.CosineSimilarity(),
                tf.keras.metrics.LogCoshError(), tfa.metrics.RSquare(y_shape=(1,))]

metrics_train_loss = []
metrics_test_loss = []

for loss in loss_list:
    metrics_loss = nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                      loss=loss, metrics=metrics_list, seed_list = range(5))
    print(f'{loss.name} - {metrics_loss}\n')
    metrics_train_loss.append(metrics_loss[0])
    metrics_test_loss.append(metrics_loss[1])

mean_squared_error - ({'mean_squared_error': 68.65995635986329, 'mean_absolute_error': 5.95288667678833, 'root_mean_squared_error': 8.286011695861816, 'mean_absolute_percentage_error': 28.699048233032226, 'mean_squared_logarithmic_error': 0.10726268738508224, 'cosine_similarity': 1.0, 'logcosh': 5.303109836578369, 'r_square': 0.22275149822235107}, {'mean_squared_error': 55.51523056030273, 'mean_absolute_error': 5.494384670257569, 'root_mean_squared_error': 7.447188186645508, 'mean_absolute_percentage_error': 29.95516929626465, 'mean_squared_logarithmic_error': 0.10368504226207734, 'cosine_similarity': 1.0, 'logcosh': 4.852791404724121, 'r_square': 0.2619813084602356})

mean_absolute_error - ({'mean_squared_error': 73.13155364990234, 'mean_absolute_error': 5.70898962020874, 'root_mean_squared_error': 8.54955883026123, 'mean_absolute_percentage_error': 25.01278419494629, 'mean_squared_logarithmic_error': 0.10520836114883422, 'cosine_similarity': 1.0, 'logcosh': 5.07847146987915, 'r_squar

Сравним все модели на тесте с помощью метрики r2

In [327]:
sorted(zip(map(lambda x: x.name, loss_list), map(lambda x: x['r_square'], metrics_test_loss)), key=lambda i: i[1], reverse=True)

[('huber_loss', 0.2659915447235107),
 ('log_cosh', 0.2642971396446228),
 ('mean_squared_error', 0.2619813084602356),
 ('mean_absolute_error', 0.24081127643585204),
 ('mean_absolute_percentage_error', 0.2009468913078308),
 ('mean_squared_logarithmic_error', 0.17541536092758178),
 ('cosine_similarity', -4.139246463775635)]

Видно, что модели с ошибками huber_loss, log_cosh и mean_squared_error справились лучше всех. mean_absolute_error, mean_absolute_percentage_error и mean_squared_logarithmic_error справились хуже. модель с cosine_similarity вообще разошлась.